In [1]:
import cv2
import numpy as np

video = cv2.VideoCapture("raw_video_feed.mp4")

width = 320
height = 240
fps = video.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

In [2]:
while video.isOpened():

    ret, frame = video.read()
    if not ret:
        break

    ####################################################################
    # logic
    ####################################################################

    out.write(frame)


video.release()
out.release()

In [3]:
def nothing(x):
	pass

img = cv2.imread("image.jpg", 0)  # grayscale

cv2.namedWindow("Edges")
cv2.createTrackbar("Min", "Edges", 50, 500, nothing)
cv2.createTrackbar("Max", "Edges", 150, 500, nothing)

while True:
	min_val = cv2.getTrackbarPos("Min", "Edges")
	max_val = cv2.getTrackbarPos("Max", "Edges")

	edges = cv2.Canny(img, min_val, max_val)
	cv2.imshow("Edges", edges)

	if cv2.waitKey(1) & 0xFF == 27:  # ESC to quit
    		break

cv2.destroyAllWindows()


[ WARN:0@5.869] global loadsave.cpp:275 findDecoder imread_('image.jpg'): can't open/read file: check file path/integrity
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/fizzer/Documents/ENPH_353_Labs/lab2/lib/python3.10/site-packages/cv2/qt/plugins"


error: OpenCV(4.12.0) /io/opencv/modules/highgui/src/window.cpp:973: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'


: 